<a href="https://colab.research.google.com/github/francji1/01NAEX/blob/main/code/01NAEX_Exercise_10_student_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# O1NAEX Exercise 10

In [ ]:
#cat(system('sudo apt-get install -y gmp', intern=TRUE), sep = "\n")
#cat(system('sudo apt-get install -y partitions', intern=TRUE), sep = "\n")
#cat(system('sudo apt-get install -y DoE.base', intern=TRUE), sep = "\n")

list_of_packages <- c("tidyverse","MASS","nlme","lme4","lsmeans") # "agricolae","nortest","lattice","pwr",FrF2
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
missing_packages
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)


In [ ]:
#tibble::tibble(
#  Package = names(installed.packages()[,3]),
#  Version = unname(installed.packages()[,3])
#)

In [ ]:
install.packages("ggplot2")
library(ggplot2)

## Drying of beech wood planks

To investigate the effect of drying of beech wood on the humidity percentage, the following experiment was conducted. Each of 20 planks was dried in a certain period of time. Then the humidity percentage was measured in 5 depths (1,3,5,7,9) and 3 widths (1,2,3) for each plank.

**Source:** The Royal Veterinary and Agricultural University, Denmark.


**Variables:**
* plank 	 -   Numbered 1-20
* width      -   Numbered 1,2,3
* depth 	 -   Numbered 1,3,5,7,9
* humidity   -   Humidity percentage

**Number of observations:** 300 (20 planks)

**Description:**
* depth 1: 	close to the top
* depth 5: 	in the center
* depth 9: 	close to the bottom
* depth 3: 	between 1 and 5
* depth 7: 	between 5 and 9
* width 1: 	close to the side
* width 3: 	in the center
* width 2: 	between 1 and 3


**Analyze data from the Drying of beech wood planks:**

* Plot four average humidity profiles: 2 interaction plots for width and 2 for depth (done).
* Carrying out the fixed effects model analysis.
* Carry out the mixed model analysis.
* Run the post hoc analysis
* Compare the fixed parameters and use the p-value correction (TukeyHSD).
 Hint: Use function `lsmeans`  from the package `lsmeans` with `adjust="tukey`.
* Summarize results.

In [ ]:
planks <- read.table("https://raw.githubusercontent.com/francji1/01NAEX/main/data/planks.txt",header=TRUE,sep=",")
planks

In [ ]:
head(planks)


In [ ]:
planks$plank  <-factor(planks$plank)
planks$width  <-factor(planks$width)
planks$depth  <-factor(planks$depth)


# Data Visualization

In [ ]:
ggplot(planks, aes(x=width, y=humidity)) +
  geom_boxplot() +
  stat_summary(fun.y="mean")

ggplot(planks, aes(x=depth, y=humidity)) +
  geom_boxplot() +
  stat_summary(fun.y="mean")

In [ ]:
# interaction plots
op <- par(mfrow=c(2,2))
with(planks, interaction.plot(width,plank,humidity,legend=T))
with(planks, interaction.plot(depth,plank,humidity,legend=T))
with(planks, interaction.plot(width,depth,humidity,legend=T))
with(planks, interaction.plot(depth,width,humidity,legend=T))
par(op)

In [ ]:
planks_with_mean <- planks %>%
  group_by(plank,width) %>%
  mutate(mean_humidity_pw = mean(humidity))%>%
  group_by(plank,depth)%>%
  mutate(mean_humidity_pd = mean(humidity))%>%
  group_by(depth,width)%>%
  mutate(mean_humidity_dw = mean(humidity))

head(planks_with_mean)

In [ ]:
# interaction plots using ggplot2
ggplot(planks_with_mean, aes(x = width, y = mean_humidity_pw, color = plank, group = plank)) +
  geom_point(position = position_dodge(width = 0.5), size = 3) +
  geom_line(position = position_dodge(width = 0.5), size = 1) +
  labs(title = "Mean Humidity for Each Plank",
       x = "Width",
       y = "Mean Humidity",
       color = "Plank") +
  theme_minimal()

ggplot(planks_with_mean, aes(x = depth, y = mean_humidity_pd, color = plank, group = plank)) +
  geom_point(position = position_dodge(width = 0.5), size = 3) +
  geom_line(position = position_dodge(width = 0.5), size = 1) +
  labs(title = "Mean Humidity for Each Plank",
       x = "Depth",
       y = "Mean Humidity",
       color = "Plank") +
  theme_minimal()

ggplot(planks_with_mean, aes(x = width, y = mean_humidity_dw, color = depth, group = depth)) +
  geom_point(position = position_dodge(width = 0.5), size = 3) +
  geom_line(position = position_dodge(width = 0.5), size = 1) +
  labs(title = "Mean Humidity for Each Plank",
       x = "Width",
       y = "Mean Humidity",
       color = "Depth") +
  theme_minimal()

ggplot(planks_with_mean, aes(x = depth, y = mean_humidity_dw, color = width, group = width)) +
  geom_point(position = position_dodge(width = 0.5), size = 3) +
  geom_line(position = position_dodge(width = 0.5), size = 1) +
  labs(title = "Mean Humidity for Each Plank",
       x = "Depth",
       y = "Mean Humidity",
       color = "Width") +
  theme_minimal()

From the interaction plots it can be seen that the mean of humidity at depth 3, 5 and 7 is higher than close to the top or bottom (depth 1 an 9).

In [ ]:
#lattice::xyplot(humidity~width | plank, groups=plank, data=planks, type=c('p','r'), auto.key=F)
#lattice::xyplot(humidity~depth | plank, groups=plank, data=planks, type=c('p','r'), auto.key=F)

# Fixed effects model

Our fixed effects are variables *depth* and *width*.

In [ ]:
attach(planks)

In [ ]:
fixed_model <- lm(humidity ~ width * depth)
summary(fixed_model)

We can see that the *depth3*, *depth5* and *depth7* are statistically significant. None of the interactions are significant so we drop them from the model.

In [ ]:
par(mfrow = c(2,2))
plot(fixed_model)

In [ ]:
fixed_model2 <- lm(humidity~ width + depth)
summary(fixed_model2)

fixed_model3 <- lm(humidity ~ depth)
summary(fixed_model3)

In [ ]:
par(mfrow = c(2,2))
plot(fixed_model2)

# Mixed model

Variable *plank* is our random effect.

In [ ]:
mixed_model <- lme(humidity ~ width *depth, random = ~ 1 | plank)
summary(mixed_model)

Random intercept for the grouping variable *plank* has standard deviation 0.9898 so variable **plank ~** $ N(0; 0.9898^2)$. As in previous case there are no significant interactions. The coeficients of fixed effects in this mixed model are the same as in the fixed model.

In [ ]:
#mixed_model_b <- lmer(humidity ~ width*depth + (1 | plank))
#summary(mixed_model_b)

In [ ]:
plot(mixed_model)

# Extract residuals from the mixed model
residuals_mixed <- residuals(mixed_model)


# Create Q-Q Residuals plot
qqnorm(residuals_mixed)
qqline(residuals_mixed, col = 2)

In [ ]:
mixed_model2 <- lmer(humidity ~ depth+width + (1 | plank))
summary(mixed_model2)

plot(mixed_model2)

# Extract residuals from the mixed model
residuals_mixed <- residuals(mixed_model2)

# Create Q-Q Residuals plot
qqnorm(residuals_mixed)
qqline(residuals_mixed, col = 2)

In [ ]:
anova(mixed_model2,fixed_model2)

In [ ]:
mixed_model <- lme(humidity ~ width * depth, random = ~ 1 | plank)
mixed_model2 <- lme(humidity ~ depth + width, random = ~ 1 | plank)

# Compare the models using anova from lmerTest
anova(mixed_model, mixed_model2)

This low p-value suggests that there is a significant difference between these two models. According to AIC and BIC the mixed-effects model is considered significantly better than the linear model in explaining the variance in the data.

In [ ]:
fixed_effects <- fixef(mixed_model2)

# Use lsmeans to compute pairwise comparisons with Tukey's HSD correction
lsmeans_results <- lsmeans(mixed_model2, specs = ~ width + depth, adjust = "tukey")

# Display the results
print(lsmeans_results)

# Plot the results
plot(lsmeans_results)

In [ ]:
install.packages("emmeans")
library(lme4)
library(emmeans)

# Assuming you have a model called mixed_model2
mixed_model2 <- lmer(humidity ~ depth + width + (1 | plank))

# Calculate the estimated marginal means
emm <- emmeans(mixed_model2, ~ depth + width)

# Pairwise comparisons with Tukey's HSD correction
pairs <- pairs(emm, adjust = "tukey")

# View the results
summary(pairs)

In [ ]:
# Calculate the estimated marginal means
emm_depth <- emmeans(mixed_model2, ~ depth)
plot(emm_depth)

In [ ]:
# Calculate the estimated marginal means
emm_width <- emmeans(mixed_model2, ~ width)
plot(emm_width)

# Summary

In this analysis, a dataset of plank measurements, including width, depth, and humidity, was explored using both fixed-effects and mixed-effects models. According to data visualization the variable depth seems to be significant. After carrying out the fixed-effects analysis, it was determined that the best model for explaining humidity includes both width and depth as independent variables. The interactions are not statistically significant. Performing a mixed-effects analysis with plank as a random effect improved the results. The mixed effect model significantly better fits the data and is better in explaining the variance in the data. The random effect *plank* has the Normal distribution $N(0;0.9797)$.